In [1]:
import cobra.test
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model
from cobra.sampling import sample
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from cobra.sampling import OptGPSampler
from scipy.stats import wilcoxon

In [2]:
## code adapated from midnighter on github https://github.com/opencobra/cobrapy/issues/771
def metabolite_flux_balance(metabolite, solution):
    """
    Return a vector of reaction fluxes scaled by the stoichiometric coefficient.

    Parameters
    ----------
    metabolite : cobra.Metabolite
        The metabolite whose fluxes are to be investigated.
    solution : cobra.Solution
        Solution with flux values.

    Returns
    -------
    pandas.Series
        A vector with fluxes of reactions that consume or produce the given
        metabolite scaled by the corresponding stoichiometric coefficients. The
        reaction identifiers are given by the index.
    """
    rxn_ids = list()
    rxn_sub = list()
    adj_flux = list()
    for rxn in metabolite.reactions:
        coef = rxn.get_coefficient(metabolite)
        rxn_ids.append(rxn.id)
        rxn_sub.append(rxn.subsystem)
        adj_flux.append(coef * solution.fluxes[rxn.id])
    return pd.DataFrame({'Rxn_id': rxn_ids, 'Subsystem': rxn_sub, 'Adjusted_Flux': adj_flux})

In [20]:
model_n2_highO2 = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")
model_n2_lowO2 = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")

In [21]:

model_n2_highO2.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_highO2.reactions.get_by_id("EX_glc__D_e").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("EX_glc__D_e").upper_bound = 0

model_n2_highO2.reactions.get_by_id("EX_sucr_e").lower_bound = -9
model_n2_highO2.reactions.get_by_id("EX_sucr_e").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("EX_sucr_e").lower_bound = -4
model_n2_lowO2.reactions.get_by_id("EX_sucr_e").upper_bound = 0


model_n2_highO2.reactions.get_by_id("ATPM").lower_bound = 110
model_n2_highO2.reactions.get_by_id("ATPM").upper_bound = 110
model_n2_lowO2.reactions.get_by_id("ATPM").lower_bound = 16
model_n2_lowO2.reactions.get_by_id("ATPM").upper_bound = 16


model_n2_highO2.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_highO2.reactions.get_by_id("O2tpp").upper_bound = 1000
model_n2_lowO2.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("O2tpp").upper_bound = 1000

model_n2_highO2.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_highO2.reactions.get_by_id("NADH6").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("NADH6").upper_bound = 0

model_n2_highO2.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_highO2.reactions.get_by_id("CYOO2pp").upper_bound = 0
model_n2_lowO2.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_lowO2.reactions.get_by_id("CYOO2pp").upper_bound = 0

In [22]:
solution_model_n2_highO2 = model_n2_highO2.optimize()

u = solution_model_n2_highO2.objective_value
DT = 0.69314718056/u


#Output growth rate
print('High O2 sucrose groowth Rate: ' ,'%.3f' %u, ' 1/h' ' or a doubling time of ' ,'%.3f' %DT, 'hrs')
model_n2_highO2.summary()

High O2 sucrose groowth Rate:  0.201  1/h or a doubling time of  3.450 hrs


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001046,0,0.00%
cl_e,EX_cl_e,0.001046,0,0.00%
cobalt2_e,EX_cobalt2_e,5.023E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001425,0,0.00%
fe2_e,EX_fe2_e,0.001658,0,0.00%
fe3_e,EX_fe3_e,0.001569,0,0.00%
h_e,EX_h_e,0.2507,0,0.00%
k_e,EX_k_e,0.03922,0,0.00%
mg2_e,EX_mg2_e,0.001743,0,0.00%
mn2_e,EX_mn2_e,0.0001388,0,0.00%


In [24]:
#Run script to determine flux to each metabolite
atp_flux = metabolite_flux_balance(model_n2_highO2.metabolites.atp_c, solution_model_n2_highO2 )
atp_flux.to_csv("../Outputs/Flux_allocation/ATPFlux_n2_high_O2.csv")

nadh_flux = metabolite_flux_balance(model_n2_highO2.metabolites.nadh_c, solution_model_n2_highO2 )
nadh_flux.to_csv("../Outputs/Flux_allocation/nadhFlux_n2_high_O2.csv")

nadph_flux = metabolite_flux_balance(model_n2_highO2.metabolites.nadph_c, solution_model_n2_highO2 )
nadph_flux.to_csv("../Outputs/Flux_allocation/nadphFlux_n2_high_O2.csv")

In [25]:
solution_model_n2_lowO2 = model_n2_lowO2.optimize()

u = solution_model_n2_lowO2.objective_value
DT = 0.69314718056/u


#Output growth rate
print('low O2 sucrose groowth Rate: ' ,'%.3f' %u, ' 1/h' ' or a doubling time of ' ,'%.3f' %DT, 'hrs')
model_n2_lowO2.summary()

low O2 sucrose groowth Rate:  0.219  1/h or a doubling time of  3.158 hrs


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001142,0,0.00%
cl_e,EX_cl_e,0.001142,0,0.00%
cobalt2_e,EX_cobalt2_e,5.487E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001556,0,0.00%
fe2_e,EX_fe2_e,0.001811,0,0.00%
fe3_e,EX_fe3_e,0.001714,0,0.00%
h_e,EX_h_e,0.2738,0,0.00%
k_e,EX_k_e,0.04284,0,0.00%
mg2_e,EX_mg2_e,0.001904,0,0.00%
mn2_e,EX_mn2_e,0.0001517,0,0.00%


In [26]:
#Run script to determine flux to each metabolite
atp_flux = metabolite_flux_balance(model_n2_lowO2.metabolites.atp_c, solution_model_n2_lowO2 )
atp_flux.to_csv("../Outputs/Flux_allocation/ATPFlux_n2_low_O2.csv")

nadh_flux = metabolite_flux_balance(model_n2_lowO2.metabolites.nadh_c, solution_model_n2_lowO2 )
nadh_flux.to_csv("../Outputs/Flux_allocation/nadhFlux_n2_low_O2.csv")

nadph_flux = metabolite_flux_balance(model_n2_lowO2.metabolites.nadph_c, solution_model_n2_lowO2 )
nadph_flux.to_csv("../Outputs/Flux_allocation/nadphFlux_n2_low_O2.csv")